In [31]:
import pandas as pd

In [61]:
europe_yearly_full_release = pd.read_csv('C:/Users/Tamás/Desktop/ESMT/FirstSem/DataVisualization/GroupProject/europe_yearly_full_release_long_format.csv')

In [ ]:
europe_yearly_full_release.info()

In [62]:
europe_yearly_full_release_copy = europe_yearly_full_release.copy()

In [34]:
europe_yearly_full_release_copy['Area'].nunique()

38

RESHAPE THE DATASET

In [35]:
# Define the columns that should appear only once (index columns)
columns_to_once = ['Area', 'ISO 3 code', 'Year', 'Area type', 'Continent', 'Ember region', 'EU', 'OECD', 'G20', 'G7', 'ASEAN']

In [36]:
# Create a unique identifier for each combination of Category, Subcategory, Variable, and Unit
europe_yearly_full_release_copy['Variable_ID'] = europe_yearly_full_release_copy.apply(
    lambda x: f"{x['Category']}_{x['Subcategory']}_{x['Variable']}_{x['Unit']}", 
    axis=1
)

In [37]:
# Pivot the DataFrame
europe_yearly_full_release_reshaped = europe_yearly_full_release_copy.pivot_table(
    index = columns_to_once,
    columns = 'Variable_ID',
    values = 'Value',
    aggfunc = 'first'
).reset_index()

In [38]:
# Reset column names from MultiIndex to single level
europe_yearly_full_release_reshaped.columns.name = None

In [ ]:
europe_yearly_full_release_reshaped.info()

In [20]:
europe_yearly_full_release_reshaped.to_csv('C:/Users/Tamás/Desktop/ESMT/FirstSem/DataVisualization/GroupProject/europe_yearly_full_release_wide_format.csv', index=False)

NEW TRY

In [95]:
europe_yearly_full_release_copy_2 = europe_yearly_full_release_copy.copy()

In [96]:
# Define the columns that should appear only once (index columns)
index_cols = ['Area', 'ISO 3 code', 'Year', 'Area type', 'Continent', 'Ember region', 
              'EU', 'OECD', 'G20', 'G7', 'ASEAN']

# Create row identifier within each year and area
europe_yearly_full_release_copy_2['row_id'] = europe_yearly_full_release_copy_2.groupby(['Year', 'Area']).cumcount() + 1


# Create wide format with suffix for each column
europe_yearly_full_release_reshaped_2 = europe_yearly_full_release_copy_2.pivot(
    index=index_cols,
    columns='row_id',
    values=['Category', 'Subcategory', 'Variable', 'Unit', 'Value']
).reset_index()

# Reset index
# Flatten the column names
europe_yearly_full_release_reshaped_2.columns = [
    f'{col[0]}_{col[1]}' if isinstance(col, tuple) else col 
    for col in europe_yearly_full_release_reshaped_2.columns
]

# Reset index
europe_yearly_full_release_reshaped_2.reset_index(drop=True, inplace=True)

In [ ]:
import re

index_cols = ['Area', 'ISO 3 code', 'Year', 'Area type', 'Continent', 'Ember region', 'EU', 'OECD', 'G20', 'G7', 'ASEAN']

cols = europe_yearly_full_release_reshaped_2.columns.tolist()
index_cols_present = [c for c in index_cols if c in cols]

# How many Category_x columns are there
pat = re.compile(r'^Category_(\d+)$')
nums = sorted({int(m.group(1)) for c in cols for m in [pat.match(c)] if m})
max_n = max(nums) if nums else 0

# Build the desired column order
ordered = index_cols_present.copy()
for n in range(1, max_n + 1):
    for base in ['Category', 'Subcategory', 'Variable', 'Unit', 'Value']:
        cname = f'{base}_{n}'
        if cname in cols:
            ordered.append(cname)

# If there are any columns not in ordered, append them at the end
remaining = [c for c in cols if c not in ordered]
ordered += remaining

# Reorder the DataFrame columns
europe_yearly_full_release_reshaped_2 = europe_yearly_full_release_reshaped_2[ordered]

In [98]:
europe_yearly_full_release_reshaped_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1285 entries, 0 to 1284
Columns: 306 entries, Category_1 to ASEAN_
dtypes: float64(5), int64(1), object(300)
memory usage: 3.0+ MB


In [101]:
europe_yearly_full_release_reshaped_2.fillna(0, inplace=True)

C:\Users\Tamás\AppData\Local\Temp\ipykernel_14576\1719143372.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  europe_yearly_full_release_reshaped_2.fillna(0, inplace=True)


In [102]:
europe_yearly_full_release_reshaped_2.to_csv('C:/Users/Tamás/Desktop/ESMT/FirstSem/DataVisualization/GroupProject/europe_yearly_full_release_wide_format_reshaped_2.csv', index=False)